BMW data cleaning and randominzation for export to csvs for MIT-NorthStar project

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import pandas as pd
import numpy as np
import calendar
import random
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import itertools
from statistics import stdev 
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

#make sure in HR environment
import argparse
import hashlib
from Crypto.Cipher import AES
from Crypto import Random
from datetime import datetime
#import pandasql as ps

#from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn import tree

# relevant tensorflow imports:

from sklearn.preprocessing import Imputer
import tensorflow as tf
from tensorflow import keras
from sklearn.utils import shuffle
# from imblearn.over_sampling import RandomOverSampler
import copy

random.seed(1)
np.random.seed(1)

/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passi

### all data from oracle connections to TS_INFO table and actuals punch data stored in HIVE

Used bmw_data_cleaning.py file to generate csvs for use here

In [2]:
df_bmw = pd.read_csv("bmw_demo.csv",
                     index_col=[0])

#cost-center headcount, gender counts, and avg tenure per month/shift/cc
cc_grp = pd.read_csv("cc_grp.csv",
                    index_col=[0])


#daily quality for assembly halls 50 and 52
quality_daily_hall = pd.read_csv("quality_daily_hall.csv")

actuals = pd.read_csv("actual_demo.csv",
                      index_col=[0])

plant_totals = pd.read_csv("plant_totals.csv")

#bmw attendance points

bmw_points = pd.read_csv("bmw_points_200114.csv")

df_ejsi = pd.read_csv("df_ejsi.csv")
pd.options.display.max_columns = None

In [3]:
plant_totals = plant_totals[plant_totals.date_month> "2018-01"]

In [4]:
df_ejsi = df_ejsi[df_ejsi.start_dt > "2019-01-01"]
df_bmw = df_bmw[df_bmw.birth_year<2010]

In [5]:
df_bmw.birth_year.max()

2001

In [6]:
df_bmw[df_bmw.birth_year>2009]

,start_date,end_date,cost_center_name,org_unit,dept,badge,status,ee_type,ta_admin_num,ta_admin_name,birth_year,gender,zip_code_type,zip_code,city,ws_rule,ws_text,daily_hours,race,last_date_worked,start_date_month,end_date_month,first_punch_cost_center,cost_center_orig,cost_center,ejsi_cc,first_ejsi_start_dt,first_ejsi_proc_desc,first_ejsi_cost_center,hall,tenure_yrs,age_at_hire,shift


In [7]:
quality_daily_hall = quality_daily_hall[quality_daily_hall.f1_date>'2018-01-01']
quality_daily_hall = quality_daily_hall.drop(quality_daily_hall.columns[0], axis=1)
quality_daily_hall.head()

,f1_date,shift,hall,volume,repair_minutes,defect_date,count_non_dp_vins_hall,rmu_hall,dpu_hall,dp_f1_hall
2592,2018-01-02,A,50,259.0,15028.5,2018-01-02,186.0,58.025097,0.718147,0.281853
2593,2018-01-02,A,52,279.0,19112.0,2018-01-02,284.0,68.501792,1.017921,-0.017921
2594,2018-01-02,B,50,323.0,10317.0,2018-01-02,255.0,31.941176,0.789474,0.210526
2595,2018-01-02,B,52,311.0,15299.0,2018-01-02,363.0,49.192926,1.167203,-0.167203
2596,2018-01-03,A,50,305.0,13044.0,2018-01-03,186.0,42.767213,0.609836,0.390164


In [8]:
# pd.crosstab(df_bmw['race'], df_bmw['gender'])

In [9]:
bmw_points.head()

,badge,name,cost_center,att_date,att_code,att_desc,hours
0,124,Tim G Howe,2666,10/10/19,9000,Daily Occurrences,8.0
1,124,Tim G Howe,2666,10/24/19,9000,Daily Occurrences,4.0
2,124,Tim G Howe,2666,10/25/19,9000,Daily Occurrences,4.0
3,139,Philip A Richards,2830,1/13/20,9000,Daily Occurrences,4.0
4,298,Edward L Kearse,1856,11/6/19,9000,Daily Occurrences,4.0


## Predict who will leave in next 30 days

For those who have already left, get the averages for the month before they left
For those still active, get averages for the month before the cutoff

In [10]:
#get the last month they were active

#df_bmw['last_date_worked'] = pd.to_datetime(np.where(df_bmw['status']=='Terminted', df_bmw['end_date'],
#                                      datetime.now().strftime("%Y-%m-%d")))
                                
#df_bmw['last_month_worked'] = df_bmw['last_date_worked'].dt.to_period('M')                             

In [11]:
df_bmw.head()
#last date worked filled into be current date as of 1-15-20

,start_date,end_date,cost_center_name,org_unit,dept,badge,status,ee_type,ta_admin_num,ta_admin_name,birth_year,gender,zip_code_type,zip_code,city,ws_rule,ws_text,daily_hours,race,last_date_worked,start_date_month,end_date_month,first_punch_cost_center,cost_center_orig,cost_center,ejsi_cc,first_ejsi_start_dt,first_ejsi_proc_desc,first_ejsi_cost_center,hall,tenure_yrs,age_at_hire,shift
0,1994-01-04,NaN,SVO,80001887,TX-453,120,Active,Hourly,A27,Todd Hill,1964,Male,Mailing Address,29645.0,Gray Court,APBC,A-Shift Paint App B/C,8,Non-Hispanic/Latino,2020-01-15,1994-01,NaN,1856.0,1856,1856.0,NaN,NaN,NaN,NaN,50.0,26.046575,29.953425,A
1,1994-01-04,2019-10-25,Machine Shop/Gun Sho,80001854,TX-212,124,Retiree,Hourly,NaN,NaN,1954,Male,Mailing Address,29369.0,moore,AICP,A-Shift ICP,8,Non-Hispanic/Latino,2019-10-25,1994-01,2019-10,2666.0,2666,2666.0,NaN,NaN,NaN,NaN,NaN,25.821918,40.178082,A
2,1994-01-04,NaN,Assem. Quality,80000650,TX-40,139,Active,Hourly,Y92,Kyle K Brooks,1970,Male,Mailing Address,29323.0,Chesnee,AFE2,A-Shift Facilities Early,10,Non-Hispanic/Latino,2020-01-15,1994-01,NaN,2830.0,2830,2830.0,NaN,NaN,NaN,NaN,50.0,26.046575,23.953425,A
3,1994-01-04,2019-10-25,Underbody/Steerng Kn,80002975,TX-436,140,Retiree,Hourly,NaN,NaN,1955,Female,Mailing Address,29302.0,Spartanburg,APBC,A-Shift Paint App B/C,8,Non-Hispanic/Latino,2019-10-25,1994-01,2019-10,1807.0,1807,1807.0,NaN,NaN,NaN,NaN,50.0,25.821918,39.178082,A
4,1994-02-21,NaN,ICP,80001886,TX-452,185,Active,Hourly,Y75,Brian Kelly Reynolds,1959,Female,Mailing Address,29681.0,Simpsonville,APBC,A-Shift Paint App B/C,8,Non-Hispanic/Latino,2020-01-15,1994-02,NaN,1900.0,1900,1900.0,NaN,NaN,NaN,NaN,NaN,25.915068,35.084932,A


## get monthly average quality KPIs


In [12]:
cc_grp.head()

#headcount and gender counts already done at month level

,shift,work_date_month,cost_center,headcount_mau_cc,headcount_bmw_cc,total_headcount_cc,count_female_cc,count_male_cc,count_nogender_cc,percentage_female_cc,avg_tenure_cc,avg_tenure_bmw_cc,count_bmw_employees_tenure,avg_tenure_mau_cc,count_mau_employees_tenure,count_hired_mau,count_hired_bmw,count_termianted_bmw,count_termianted_mau
0,A,2018-04,1026.0,67.0,5.0,72.0,22.0,39.0,0.0,0.360656,1.171542,17.191096,4.0,0.066745,57.0,NaN,NaN,NaN,NaN
1,A,2018-04,1317.0,46.0,4.0,50.0,11.0,37.0,0.0,0.229167,2.613918,11.825342,4.0,1.812924,44.0,4.0,NaN,NaN,2.0
2,A,2018-04,1319.0,26.0,4.0,30.0,12.0,36.0,1.0,0.244898,3.764000,10.542009,6.0,2.839726,43.0,NaN,NaN,NaN,1.0
3,A,2018-04,1340.0,33.0,4.0,37.0,7.0,46.0,0.0,0.132075,5.317310,15.943836,6.0,4.016103,47.0,6.0,NaN,NaN,1.0
4,A,2018-04,1341.0,8.0,0.0,8.0,1.0,4.0,0.0,0.200000,9.761644,NaN,NaN,9.761644,5.0,2.0,NaN,NaN,NaN


In [13]:
quality_daily_hall.head()

,f1_date,shift,hall,volume,repair_minutes,defect_date,count_non_dp_vins_hall,rmu_hall,dpu_hall,dp_f1_hall
2592,2018-01-02,A,50,259.0,15028.5,2018-01-02,186.0,58.025097,0.718147,0.281853
2593,2018-01-02,A,52,279.0,19112.0,2018-01-02,284.0,68.501792,1.017921,-0.017921
2594,2018-01-02,B,50,323.0,10317.0,2018-01-02,255.0,31.941176,0.789474,0.210526
2595,2018-01-02,B,52,311.0,15299.0,2018-01-02,363.0,49.192926,1.167203,-0.167203
2596,2018-01-03,A,50,305.0,13044.0,2018-01-03,186.0,42.767213,0.609836,0.390164


In [14]:
quality_daily_hall['f1_month'] = pd.to_datetime(quality_daily_hall['f1_date']).dt.to_period('M')

#sum up all volume created in that month

quality_daily_hall_monthly = quality_daily_hall.groupby([
                                                         'f1_month',
                                                            'shift',
                                                            'hall'],
                                    as_index=False).agg({
                                        'volume':'sum',
                                        'repair_minutes':'sum',
                                        'count_non_dp_vins_hall':'sum'})


quality_daily_hall_monthly['RMU_hall_month'] = quality_daily_hall_monthly['repair_minutes'] / (quality_daily_hall_monthly['volume']).astype('float')
quality_daily_hall_monthly['dpu_hall_month'] = quality_daily_hall_monthly['count_non_dp_vins_hall']/ quality_daily_hall_monthly['volume']

quality_daily_hall_monthly['f1_dp_hall_month'] = (quality_daily_hall_monthly['volume'] - quality_daily_hall_monthly['count_non_dp_vins_hall'])/ quality_daily_hall_monthly['volume']


In [15]:
quality_daily_hall_monthly.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92 entries, 0 to 91
Data columns (total 9 columns):
f1_month                  92 non-null period[M]
shift                     92 non-null object
hall                      92 non-null int64
volume                    92 non-null float64
repair_minutes            92 non-null float64
count_non_dp_vins_hall    92 non-null float64
RMU_hall_month            92 non-null float64
dpu_hall_month            92 non-null float64
f1_dp_hall_month          92 non-null float64
dtypes: float64(6), int64(1), object(1), period[M](1)
memory usage: 7.2+ KB


In [16]:
quality_daily_hall_monthly.head()

,f1_month,shift,hall,volume,repair_minutes,count_non_dp_vins_hall,RMU_hall_month,dpu_hall_month,f1_dp_hall_month
0,2018-01,A,50,8663.0,431810.0,4442.0,49.845319,0.512755,0.487245
1,2018-01,A,52,7625.0,528209.5,5900.0,69.273377,0.773770,0.226230
2,2018-01,B,50,9313.0,393935.0,5975.0,42.299474,0.641576,0.358424
3,2018-01,B,52,8274.0,507186.0,6795.0,61.298767,0.821247,0.178753
4,2018-02,A,50,9423.0,455685.5,4380.0,48.358856,0.464820,0.535180


## monthly sum of points per person

In [17]:
bmw_points.head()

,badge,name,cost_center,att_date,att_code,att_desc,hours
0,124,Tim G Howe,2666,10/10/19,9000,Daily Occurrences,8.0
1,124,Tim G Howe,2666,10/24/19,9000,Daily Occurrences,4.0
2,124,Tim G Howe,2666,10/25/19,9000,Daily Occurrences,4.0
3,139,Philip A Richards,2830,1/13/20,9000,Daily Occurrences,4.0
4,298,Edward L Kearse,1856,11/6/19,9000,Daily Occurrences,4.0


In [18]:
bmw_points['att_month'] = pd.to_datetime(bmw_points['att_date']).dt.to_period('M')

bmw_points_monthly = bmw_points.groupby(['badge', 'att_month'], as_index=False).agg({'hours':'sum'})

bmw_points_monthly.rename(columns={'hours':'att_points'}, inplace=True)
bmw_points_monthly.head()

,badge,att_month,att_points
0,124,2019-10,16.0
1,139,2020-01,4.0
2,298,2019-11,12.0
3,375,2019-05,5.0
4,397,2020-01,32.0


In [19]:
# get avg time spent on each location per person per month

df_ejsi['end_dt'] = pd.to_datetime(df_ejsi['end_dt'])
df_ejsi['start_dt'] = pd.to_datetime(df_ejsi['start_dt'])
df_ejsi['hours_at_loc'] = (df_ejsi['end_dt'] - df_ejsi['start_dt']).dt.components['days']*24\
    +(df_ejsi['end_dt'] - df_ejsi['start_dt']).dt.components['hours']+((df_ejsi['end_dt'] - df_ejsi['start_dt']).dt.components['minutes'])/60 #get amount of hours spent at each station
df_ejsi.head()

SystemError: <built-in function is_scalar> returned a result with an error set

In [ ]:
### EJSI most frequent sign in location, and avg time spent on each location per person per month

df_ejsi['month'] = pd.to_datetime(df_ejsi['start_dt']).dt.to_period('M') #start date that they were workin on that line location

df_ejsi_monthly_avg = df_ejsi.groupby(['badge', 'month'], as_index=False).agg({'hours_at_loc':['mean', 'max']})
df_ejsi_monthly_avg.columns = ['badge', 'month','avg_hours_at_line_loc', 'max_hours_at_line_loc']

df_ejsi_monthly = df_ejsi.groupby(['badge', 'month'], as_index=False)['line_loc'].agg(lambda x: pd.Series.mode(x)[0])


#join in the avg time spent at any line location
df_ejsi_monthly = df_ejsi_monthly.merge(df_ejsi_monthly_avg,
                                       on=['badge', 'month'],
                                       how='left')

In [ ]:
#df_ejsi_monthly.columns = ['badge', 'ejsi_month', 'line_loc_mode', 'avg_hours_at_loc']
df_ejsi_monthly.head()

## Actual punch in averages per person month

In [ ]:
actuals.head()
actuals['work_date'] = pd.to_datetime(actuals['work_date'])
actuals['month'] = actuals['work_date'].dt.to_period('M')

In [ ]:
actuals['description'].unique()

In [ ]:
actuals_monthly_avg = actuals[actuals['description']=='Regular Hours'].groupby(['badge', 'month'], as_index=False)\
.agg({'num_hours':['mean', 'sum']})

actuals_monthly_avg.columns = ['badge', 'month', 'avg_reg_hours', 'sum_reg_hours']
actuals_monthly_avg.head()

In [ ]:
OT_list = ['Double Time', 'DTime: Premium', 'Double time', 'DTime:Premium']


In [ ]:
actuals['month'] = pd.to_datetime(actuals['work_date']).dt.to_period('M')
#gets the most frequent cost-center
actuals_monthly = actuals.groupby(['badge', 'month'], as_index=False)['cost_center'].agg(lambda x: pd.Series.mode(x))

#avg normal hours
actuals_monthly_avg_OT = actuals[actuals.description.isin(OT_list)].groupby(['badge', 'month'], as_index=False)\
.agg({'num_hours':['mean', 'sum']})
actuals_monthly_avg_OT.columns = ['badge', 'month', 'avg_OT_hours', 'sum_OT_hours']

#total overtime hours per month, and avg hours per month
actuals_monthly_avg_reg = actuals[actuals['description']=='Regular Hours'].groupby(['badge', 'month'], as_index=False)\
.agg({'num_hours':['mean', 'sum']})
actuals_monthly_avg_reg.columns = ['badge', 'month', 'avg_reg_hours', 'sum_reg_hours']


In [ ]:
actuals_monthly_avg_reg.head()

In [ ]:
#merge the mode, or most frequent cc to the avg time clocked in per day for that month/person
actuals_monthly = actuals_monthly.merge(actuals_monthly_avg_reg,
                                       on=['badge', 'month'],
                                       how='left')

#merge in overtime
actuals_monthly = actuals_monthly.merge(actuals_monthly_avg_OT,
                                       on=['badge', 'month'],
                                       how='left')

#actuals_monthly.head()

#print(actuals_monthly.iloc[3000, actuals_monthly.columns.get_loc('avg_reg_hours')])
#print(actuals_monthly)

In [ ]:
actuals_monthly

In [ ]:
actuals_monthly_avg_reg

In [ ]:
actuals_monthly_avg_OT

# it seems that there is a lot more rows in actuals_monthly_avg_reg than in actuals_monthly_avg_OT and some badges...
    # ... are only in one

## Create profile for each person. 

For active employees, it is whatever happened the month before our chosen cutoff month.
For termianted employees, it is whatever happened the month before the month they were terminated in.

In [ ]:
# we only want to see people who were terminated after 2018-01, or who are still active

df_bmw = df_bmw[((df_bmw.status=='Terminated') & (df_bmw.end_date >= "2018-01-01"))\
               | (df_bmw.status=='Active')]

In [ ]:
df_bmw[df_bmw.end_date.isna()]

In [ ]:
df_bmw[(df_bmw.cost_center==1222) & (df_bmw.status=='Terminated')]

In [ ]:
df_bmw[df_bmw.dept=='Inactive']

# in df_bmw, remoe anybody in inactive department, as they are all terminated and are biasing data

In [ ]:
##testVar = df_bmw[df_bmw['status']=='Terminated']
##xx=df_bmw.groupby(['status','dept'],
##                 as_index=False)['ee_type'].count()
##xx[xx.dept=='Inactive']

In [ ]:
work_months =[]
start1=[]
end1=[]
df_list = []

for index, row in df_bmw.iterrows():
    if df_bmw.loc[index, 'start_date'] < '2018-01-01':
        start_date = '2018-01-01'
    else:
        start_date = df_bmw.loc[index, 'start_date']
    #end_date = df_bmw.loc[index, 'last_date_worked']
    months_worked = pd.date_range(start_date , df_bmw.loc[index, 'last_date_worked'], freq='MS').strftime("%Y-%m").tolist()
    
    #create dataframe for each person
    df = pd.DataFrame(months_worked)
    df['badge'] = df_bmw.loc[index, 'badge']
    #df.columns = ['month', 'badge']
    
    work_months.append(months_worked)
    df_list.append(df)

In [ ]:
#len(work_months) #list
#work_months[1]

In [ ]:
#make it into 1 dataframe
df_work_months = pd.concat(df_list)
df_work_months.columns = ['month', 'badge']

In [ ]:
#merge in the demographics for each person
bmw_demo = df_bmw.drop(['ta_admin_num',
                       'ta_admin_name', #terminated ppl do not have a ta_admin, so it would bias model
                       'zip_code_type',
                       'last_date_worked',
                       'end_date_month',
                        'first_ejsi_start_dt',
                        'first_ejsi_proc_desc'],
                        axis=1
                       )

df_work_months['month'] = df_work_months['month'].astype('str')

employee_matrix = df_work_months.merge(bmw_demo,
                         left_on=['badge'],
                         right_on=['badge'],
                         how='left')

#redo tenure for each month

employee_matrix['tenure_yrs'] = ((pd.to_datetime(employee_matrix['month']+'-01')\
                                - pd.to_datetime(employee_matrix['start_date'])).dt.days)/365

employee_matrix

In [ ]:
#merge in the bmw_points_monthly for each person
bmw_points_monthly['att_month'] = bmw_points_monthly['att_month'].astype('str')

employee_matrix = employee_matrix.merge(bmw_points_monthly,
                         left_on=['badge', 'month'],
                         right_on=['badge', 'att_month'],
                         how='left')

In [ ]:
#merge in quality based on hall
quality_daily_hall_monthly['f1_month'] = quality_daily_hall_monthly['f1_month'].astype('str')

employee_matrix = employee_matrix.merge(quality_daily_hall_monthly,
                         left_on=['hall', 'shift', 'month'],
                         right_on=['hall', 'shift', 'f1_month'],
                         how='left')

In [ ]:
actuals_monthly

In [ ]:
# I suspect that it only has the values in the correct months, the data is probably fine for the sum and avg columns
    # not gonna waste any more time looking at these

In [ ]:
# Merge in actual hours worked each month

employee_matrix = employee_matrix.merge(actuals_monthly.drop('cost_center', axis=1),
                                        on=['badge', 'month'],
                                        how='left')
#all quality data is for dates up to that month end, ex if month is 2019-03, 
#the qulity data is aggregated for 2019-03-01 to 2019-03-31

employee_matrix.head()

In [ ]:
# Merge in df_ejsi_monthly

employee_matrix = employee_matrix.merge(df_ejsi_monthly,
                                       on=['badge','month'],
                                       how='left')

## Label if that person left next month or not

In [ ]:
employee_matrix['next_month'] = (pd.to_datetime(employee_matrix['month']+'-01')+timedelta(days=31)).dt.to_period('M')
employee_matrix['next_month2'] = (pd.to_datetime(employee_matrix['month']+'-01')+timedelta(days=62)).dt.to_period('M')

employee_matrix['label']= np.where((pd.to_datetime(employee_matrix['end_date']) >= pd.to_datetime(employee_matrix['next_month'].astype('str')+'-01'))\
                                   & (pd.to_datetime(employee_matrix['end_date']) <= pd.to_datetime(employee_matrix['next_month2'].astype('str')+'-01')),
                                   1,
                                   0)


#remove that last entry whih is in the month that they left
#since we are using the data for that month (ex month 2019-03 includes data from 2019-03-01 to 2019-03-31), 
# and that person termianted on 2019-04-10, for the month 2019-04 technically they were employeed for part of it, 
# but since they leave midway through we cannot use the enitre month of data to predict if they are going to leave
employee_matrix = employee_matrix[(pd.to_datetime(employee_matrix['end_date'])) >= (pd.to_datetime(employee_matrix['next_month'].astype('str')+'-01'))]


In [ ]:
employee_matrix

## In below two boxes, we are attempting to run model with only a few columns to simplify an adding back as we go:

In [ ]:
# drop columns that we don't want:
    # months only used for making slices, don't actually need them

#employee_matrix.drop(['status','cost_center_name','city','ws_text','first_punch_cost_center',
#                      'dept','cost_center_orig','first_ejsi_cost_center', 'end_date',
#                     'f1_month','next_month','next_month2','avg_reg_hours','sum_reg_hours',
#                     'avg_OT_hours','sum_OT_hours','month','start_date'], axis=1, inplace=True)


# dropping from bmw_demo:
# employee_matrix.drop(['start_date','start_date_month','end_date','cost_center_name','org_unit','dept','status',
#                       'birth_year','ee_type','first_ejsi_cost_center','f1_month','ws_rule','ws_text','att_month',
#                       'cost_center_orig','ejsi_cc','first_punch_cost_center','month',
#                      'next_month','next_month2'], axis=1, inplace=True)

# dropping from points:
#employee_matrix.drop(['name','att_date','att_desc'], axis=1, inplace=True)

# dropping from actuals:
# employee_matrix.drop(['work_date','category','description','ws_rule','ws_rule_text','status'], axis=1, inplace=True)

# dropping from quality_daily_hall:
# employee_matrix.drop(['f1_date','defect_date','count_non_dp_vins_hall'], axis=1, inplace=True)


## trying employee_matrix with only a few of the columns for simplicity and debugging:

employee_matrix = employee_matrix[['badge','label','att_points','cost_center','gender','race','daily_hours','city','avg_OT_hours','avg_reg_hours']]

In [ ]:
# # for i in range(len(employee_matrix.att_points)):
# #     if employee_matrix.att_points

# print(employee_matrix['att_points'])

In [ ]:
# one-hot encoding:


# ensuring all cols with strings are one-hot encoded - can deal with each non-string one hot col manually
    # making list of cols with strings and passing to get_dummies function (oneHotList)
    
oneHotList = []

#print(employee_matrix.badge)
print(type(employee_matrix.badge.iloc[0]))


# part of the below if statement???:
# not (np.isnan(employee_matrix[feature].iloc[0])) and 
for feature in employee_matrix.columns:
    #print('feature is: ', feature)
    # do nan values have a type?
        # first part of if statement to short-circuit just in case they don't
    # checking first value in a column, if it is a string or not
    if type(employee_matrix[feature].iloc[0]) is str:
        oneHotList.append(feature)
        
# add all cols to be one-hot-encoded that do not have strings manually:

#oneHotList.extend(('cost_center','hall','daily_hours','zip_code'))


# use line above for final:

oneHotList.extend(('cost_center','daily_hours'))

#print('oneHotList is: ', oneHotList)
# employee_matrix = pd.get_dummies(data=employee_matrix, columns = ['cost_center','gender','race','hall','shift',
#                                                          'daily_hours','zip_code','line_loc','city'], drop_first = True)

employee_matrix = pd.get_dummies(data=employee_matrix, columns=oneHotList, drop_first = True)

# make sure to one-hot encode line_loc from employee_matrix!!

# difference in value between badge numbers is not important, guy with badge number 40 is 2x better than 20
    # the specific badge number has no heeding if you're fired
    
# employee_matrix = oneHot.fillna(0)

# iterate through all columns in df, if a singline na value in a column, fill all na's with mean from that column
    # imputing withiout sklearn
    
#featureArr = np.array(employee_matrix.columns)

#for feature in featureArr:
#    print('old column is: ')
#    print('\n')
#    print(employee_matrix[feature])
#    employee_matrix[feature].fillna(employee_matrix[feature].mean(), inplace=True)
#    print('\n')
#    print('new column is: ')
#    print('\n')
#    print(employee_matrix[feature])
#employee_matrix = employee_matrix.fillna(0)

#employee_matrix.to_csv("trial_one_hot_encoding_employee_matrix.csv")

# imputing with sklearn:


# try 4 sklearn:

#print(employee_matrix.sum_OT_hours)
# imputer function removes all columns that only have NAN.. for some reason some of our columns have only NAN
    # get rid of them before imputing

# removing all columns that have ONLY NAN values:
employee_matrix=employee_matrix.dropna(axis=1, how='all')

# fill att_points with 0, not mean
employee_matrix['att_points'].fillna(0, inplace=True)

print(employee_matrix['att_points'])

fill_NaN = Imputer(missing_values=np.nan, strategy='mean', axis=0)
imputed_employee_matrix= pd.DataFrame(fill_NaN.fit_transform(employee_matrix))
#print(len(employee_matrix.columns))
#print(len(imputed_employee_matrix.columns))
#employee_matrix.head()
#imputed_employee_matrix.head()

imputed_employee_matrix.columns = employee_matrix.columns
imputed_employee_matrix.index = employee_matrix.index

employee_matrix = imputed_employee_matrix


# try 2 sklearn:

#featureList = list(employee_matrix.columns)

#print(featureList)

#imputer = Imputer(missing_values ='NaN', strategy = 'mean', axis = 0)

#q=imputer.fit_transform(employee_matrix[featureList]).T

#employee_matrix[featureList]=q


# try 3 sklearn

#imp=Imputer(missing_values="NaN", strategy="mean" )

#employee_matrix[featureList]=imp.fit_transform(employee_matrix[[featureList]]).ravel()

In [ ]:
#employee_matrix.head()


#print(np.array(employee_matrix.columns))
#roc = np.array(employee_matrix.columns)
#print(type(roc))
#print(type(roc[5]))
#employee_matrix

In [ ]:
## TRIAL:

shuff_try = [employee_matrix for _, employee_matrix in employee_matrix.groupby('badge')]
random.shuffle(shuff_try)

#print(shuff_try)

employee_matrix = pd.concat(shuff_try)
#print(type(employee_matrix))
#print(employee_matrix)
#employee_matrix.to_csv('trial_shuffle.csv')

In [ ]:
print(type(employee_matrix))

In [ ]:
employee_matrix

## 3 month slices of peoples history

In [ ]:
# could dictionaries based on badge here as well as in the creation of employee matrix be quicker/more efficient?

# later: - MAKE SURE TO DELETE DATA WITH LESS THAN 4 MONTHS WORKED 
employee_slices = []
label_list=[]

print(type(employee_matrix))
for badge in employee_matrix['badge'].unique():
    df = employee_matrix[employee_matrix.badge==badge]
    #print(df)
    #print(df)
    df_length = len(df)
    if df_length >= 4:
        # get rid of people who worked less than 4 months and don't have enough data to predict if they left or not.
        for i in range(df_length - 2): #get slices of 3 months
            df_slice = (df[i: i+3])
            employee_slices.append(df_slice)
            label = df_slice.iloc[-1]['label']
            label_list.append(label)
        
# print(type(employee_slices[0]))

In [ ]:
#print(employee_matrix[employee_matrix.badge==badge])
#print(employee_matrix.badge==badge)

In [ ]:
# open jupyter in terminal with the below line so that this cell will run properly:
# jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000

# first try not flattening it, if we want to flatten, just had .flatten() to the end of .to_numpy():

# drop label and badge column from all df's in train and test:
    
X_input = [None for i in range(len(employee_slices))]

#shuff_try = [employee_matrix for _, employee_matrix in employee_matrix.groupby('badge')]
#random.shuffle(shuff_try)

# X_input, y_input = shuffle(X_input, y_input, random_state=0)



# Try upsampling Natalie's way:



# don't drop badge here for now:
for i in range(len(employee_slices)):
    employee_slices[i] = employee_slices[i].drop(['label'], axis=1)
    
for i in range(len(employee_slices)):
    X_input[i] = employee_slices[i].values
    
    
X_input = np.array(X_input)
print(X_input)

In [ ]:
# making y_train:
y_input = np.array(label_list)
#print(len(label_list))
#print(y_train)

In [ ]:
# No need to shuffle these here:
# randomize employee_slices and label_list with same seed so that all 1.0's aren't first and up in training set:
    # Also, make sure that same people are also in training or test only, or this could get messy
        # how will that be possible????!! - perhaps it has to happen above
#X_input, y_input = shuffle(X_input, y_input, random_state=0)

In [ ]:
# Creating testing data (train/test split):

# 80% of data is training, 20% is for testing
    #X_train is now 80% of its previous size, other 20% is now testing

    
# make sure to not have same person in training and test data!

    # pick a percentage of badges at random (80%)
    # don't want only older people in training, newer in testing
    
train_size = int(len(X_input) * 0.80)
test_size = len(X_input) - train_size

X_train, X_test = X_input[0:train_size], X_input[train_size:len(X_input)]

y_train, y_test = y_input[0:train_size], y_input[train_size:len(y_input)]
    

#print(len(y_train))
#print(len(X_train))
#print(len(y_test))
#print(len(X_test))

# print(X_train)
# print('\n')
# print('\n')
# print('\n')
# print('\n')

# print(y_train)
# print('\n')
# print('\n')
# print('\n')
# print('\n')

# print(X_test)
# print('\n')
# print('\n')
# print('\n')
# print('\n')

# print(y_test)
print(X_train)
print(y_train)

In [ ]:
# Manually Oversampling:
# print(X_train)
# print(y_train)
# print(type(X_train))
# print(type(X_train[0]))
# print(type(X_train[0][0]))

zeroCount = 0
oneCount = 0
for item in y_train:
    if item == 0:
        zeroCount +=1
    elif item == 1:
        oneCount +=1

print('zeroCount is: ', zeroCount)
print('oneCount is: ', oneCount)

In [ ]:
# randomly select samples (1-month slices) to be copied and appended to the end of the array

# try adding samples of 1.0 class with a while loop???

orig_y_train_len = len(y_train)

# prop is guaranteed to be greater than or equal to the proportion zeroCount/OneCount

prop = (zeroCount//oneCount)+1    
# print(prop)

# prop = 2

# going through X_train, and retrieving all samples that are of the minority (1) class:
    #make list first, convert to numpy array once at desired size

onesOnlyTrainX = []
onesOnlyTrainY = []

# don't believe i need to index with i%orig_y_train_len here.... just want all examples of 1
for i in range(orig_y_train_len):
    if y_train[i] == 1:
        onesOnlyTrainX.append(X_train[i])
        onesOnlyTrainY.append(y_train[i])
        
print('length onesOnlyTrainY is: ', len(onesOnlyTrainY))
print('length onesOnlyTrainX orig is: ', len(onesOnlyTrainX))

In [ ]:
# print(onesOnlyTrainX)

# convert to arrays from lists
onesOnlyTrainX = np.array(onesOnlyTrainX)
onesOnlyTrainY = np.array(onesOnlyTrainY)

# want to add the ORIGINAL onesOnlyTrainX, so we add same amount every time and doesn't get exponential:
onesOnlyTrainXOrig = copy.deepcopy(onesOnlyTrainX)
onesOnlyTrainYOrig = copy.deepcopy(onesOnlyTrainY)

# append onesOnlyTrainX to itself prop times: (dynamic)
    # len(onesOnlyTrainX) guaranteed to be greater than or equal to zeroCount due to prop
for i in range(prop):
    onesOnlyTrainX = np.vstack((onesOnlyTrainX, onesOnlyTrainXOrig))
    onesOnlyTrainY = np.append(onesOnlyTrainY, onesOnlyTrainYOrig)
    
print('length onesOnlyTrainX Final: ', len(onesOnlyTrainX))
print('length onesOnlyTrainY Final: ', len(onesOnlyTrainY))

In [ ]:
# shuffle after appending them:
    # ensuring they are shuffled in same order

indices = np.arange(onesOnlyTrainY.shape[0])
np.random.shuffle(indices)

onesOnlyTrainY = onesOnlyTrainY[indices]
onesOnlyTrainX = onesOnlyTrainX[indices]

In [ ]:
# no need for modular arithmetic:

# going through the list of shuffled months with corresponding 1.0 category, and adding them to X_train until ...
    #... same amount of both categories
i = 0
while oneCount < zeroCount:
        X_train = np.vstack((X_train, [onesOnlyTrainX[i]]))
        # also make sure to change y_train accordingly to still match with X_train:
        y_train = np.append(y_train, onesOnlyTrainY[i])
        
        oneCount+=1
        i+=1
        #print('i is: ', i)
        #print('oneCount is: ', oneCount)

In [ ]:
# perhaps shuffle X_train and y_train again after creating them???? probably not helpful...
indices = np.arange(X_train.shape[0])
np.random.shuffle(indices)

X_train = X_train[indices]
y_train = y_train[indices]

In [ ]:
# ensuring that same amount of majority and minority class in each:
oneCountFinal = 0
zeroCountFinal = 0
for item in y_train:
    if item == 0:
        zeroCountFinal +=1
    elif item == 1:
        oneCountFinal +=1

print(oneCountFinal)
print(zeroCountFinal)
#print(X_train)
print(len(X_train))
print(len(y_train))

In [ ]:
# print(X_train)
# print(X_train.shape[0])
# print(X_train.shape[1])
# print(X_train.shape[2])
print(X_train)
print('\n')
print(y_train)

plt.hist(y_train)

# badge in first column

In [ ]:
print(X_train.shape)

In [ ]:
# drop badge right before we train the model:
    # have to iterate through both train and test and drop
        # X_train y_train, X_test, y_test
    

# drop badge from training data:
    # is it the first column??
# for i in range(len(y_train)):
#     for j in range(len(y_train[i])):
        
#     employee_slices[i] = employee_slices[i].drop(['badge'], axis=1)

X_train = X_train[:, :, 1:]
# y_train = y_train[1:]

X_test = X_test[:, :, 1:]
# y_test = y_test[1:]

print(X_train)
print('\n')
print('\n')
print(y_train)


# checking that types are the same:
print(X_train.shape)

In [ ]:
loss = 'binary_crossentropy'

# if loss == 'sparse_categorical_crossentropy':
#     for i in range(len(y_test)):
#         if y_test[i][0] == 1:
#             y_test[i][0] = 0.99
    
#     for j in range(len(X_test)):
#         if X_test[j][0] == 1:
#             X_test[j][0] = 0.99


print('training model')
print('\n')

# Actually building the net:

# print(X_train.shape[0])
# print(X_train.shape[1])
# print(X_train.shape[2])

model = keras.Sequential()
model.add(keras.layers.LSTM(
  units=31,
    return_sequences = True,
  input_shape=(X_train.shape[1], X_train.shape[2])
))
# change return_sequences to true if adding additional LSTM layer

model.add(keras.layers.LSTM(units=250, return_sequences = True))
model.add(keras.layers.LSTM(units=1000))
# model.add(keras.layers.LSTM(units=1500))
model.add(keras.layers.Dense(activation='sigmoid',units=1))
model.compile(
  loss= loss,
    # try categorical crossentropy
  optimizer=keras.optimizers.Adam(lr=0.0001),
  metrics=['accuracy']  
)

# class_weight= {0:10.,
#                1:1.}

history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=1,
    #class_weight=class_weight,
    verbose=1,
    shuffle=False
)

# try adding another LSTM layer - might increase accuracy - tried
# adjust loss and accuracy functiond - tried
# try weights with no upsampling - tried
    # try weightd with Upsampling - tried, no effet
# google ways to increase precision


In [ ]:
# print(model.metrics_names)

In [ ]:
model.evaluate(X_test,y_test,verbose=2)
# second line of output is just (loss, accuracy) I believe

In [ ]:
# PREDICTION: Testing the results of our model:

## Predicting one value:

# emptyList = [0]
# emptyList[0] = X_test[0]

# emptyArray = np.array(emptyList)

# y_pred = model.predict(emptyArray)

#print(y_pred)


## Predicting from entire X_test:

y_pred3 = model.predict(X_test)

print(y_pred3)

# the probability is the probability that they will get fired in the next month from a three month segment

In [ ]:
## Predicting the results for the entire test dataset:

#y_pred2 = model.predict(X_test)
#print("y_pred2 is: ", y_pred2)

# A random predicted value in the dataset:

#print("y_pred2[32] is: ", y_pred2[32])


# compare y_test with the prediction from X_test in above cell:
#print(y_test)
#print(type(y_test[5]))
print(y_train)

In [ ]:
# checking accuracy of model for each class, separately:

# Y_test = np.argmax(y_test) # Convert one-hot to index

y_pred4 = model.predict_classes(X_test)

print(classification_report(y_test, y_pred4))

# confusion matrix:
confusion_matrix(y_test, y_pred4, labels=[0,1])

# remember that X_test does not need to be balanced, only X_train
    # the fact that X_test isn't balanced doesn't affect our results at all
    

In [ ]:
sum(y_pred4)
#print(len(y_pred4))

In [ ]:
sum(y_test)

In [ ]:
# summary of the model:
model.summary()